# TruBot

In [1]:
%load_ext autoreload
%autoreload 2
from pathlib import Path
import sys

sys.path.append(str(Path().cwd().parent.parent.resolve()))

In [3]:
from IPython.display import JSON

# Imports main tools:
from trulens_eval import Feedback
from trulens_eval import Huggingface
from trulens_eval import Tru
from trulens_eval import TruChain
from trulens_eval import Query

tru = Tru()

# imports from langchain to build app
from langchain.chains import LLMChain
from langchain.llms import OpenAI
from langchain.prompts.chat import ChatPromptTemplate
from langchain.prompts.chat import HumanMessagePromptTemplate
from langchain.prompts.chat import PromptTemplate
import numpy as np

full_prompt = HumanMessagePromptTemplate(
    prompt=PromptTemplate(
        template=
        "Provide a helpful response with relevant background information for the following: {prompt}",
        input_variables=["prompt"],
    )
)

chat_prompt_template = ChatPromptTemplate.from_messages([full_prompt])

llm = OpenAI(temperature=0.9, max_tokens=128)

chain = LLMChain(llm=llm, prompt=chat_prompt_template, verbose=True)

KEY SET: OPENAI_API_KEY
KEY SET: PINECONE_API_KEY
KEY SET: PINECONE_ENV
KEY SET: HUGGINGFACE_API_KEY
KEY SET: SLACK_TOKEN
KEY SET: SLACK_SIGNING_SECRET
KEY SET: COHERE_API_KEY


ImportError: cannot import name 'ChainQuery' from 'trulens_eval.tru_db' (/home/piotrm/repos/trulens/trulens_eval/trulens_eval/tru_db.py)

In [ ]:
# tc = Tru().Chain(chain=chain, chain_id="hello")
# res, record = tc.call_with_record("Who is Piotr?")

In [ ]:
from pprint import PrettyPrinter
from typing import Sequence

from IPython.display import JSON
from langchain.chains import ConversationalRetrievalChain
from langchain.embeddings.openai import OpenAIEmbeddings
from langchain.llms import OpenAI
from langchain.memory import ConversationSummaryBufferMemory
from langchain.vectorstores import DocArrayHnswSearch
import numpy as np

from trulens_eval import Tru
from trulens_eval import tru_feedback
from trulens_eval.keys import *
from trulens_eval.tru_db import Query
from trulens_eval.tru_db import RecordQuery
from trulens_eval.tru_feedback import Feedback
from trulens_eval.tru_feedback import Huggingface
from trulens_eval.utils.langchain import WithFilterDocuments

# if using Pinecone vectordb:
# from langchain.vectorstores import Pinecone
# import pinecone

pp = PrettyPrinter()

In [ ]:
model_name = "gpt-3.5-turbo"
chain_id = "TruBot"

# Pinecone configuration if using pinecone.
# pinecone.init(
#    api_key=PINECONE_API_KEY,  # find at app.pinecone.io
#    environment=PINECONE_ENV  # next to api key in console
#)
#docsearch = Pinecone.from_existing_index(
#    index_name="llmdemo", embedding=embedding
#)

# LLM for completing prompts, and other tasks.
llm = OpenAI(temperature=0, max_tokens=256)

openai = tru_feedback.OpenAI()

def new_conversation(
    lang_prompt_fix: bool = False,
    context_prompt_fix: bool = False,
    context_filter_fix: bool = False,
    feedbacks: Sequence[Feedback] = None
):
    """
    Create a chain for a new conversation (blank memory). Set flags to enable
    adjustments to prompts or add context filtering.
    """
    
    assert not(lang_prompt_fix and context_prompt_fix), "Cannot use both prompt fixes at the same time."

    # Embedding needed for Pinecone vector db.
    embedding = OpenAIEmbeddings(model='text-embedding-ada-002')  # 1536 dims

    # Conversation memory.
    memory = ConversationSummaryBufferMemory(
        max_token_limit=650,
        llm=llm,
        memory_key="chat_history",
        output_key='answer'
    )

    # Pinecone alternative. Requires precomputed 'hnswlib_truera' folder.
    docsearch = DocArrayHnswSearch.from_params(
        embedding=embedding,
        work_dir='hnswlib_trubot',
        n_dim=1536,
        max_elements=1024
    )
    retriever = docsearch.as_retriever()

    # Better contexts fix, filter contexts with relevance:
    if context_filter_fix: 
        def filter_by_relevance(query, doc):
            return openai.qs_relevance(
                question=query, statement=doc.page_content
            ) > 0.5

        retriever = WithFilterDocuments.of_retriever(
            retriever=retriever, filter_func=filter_by_relevance
        )

    # Conversational chain puts it all together.
    chain = ConversationalRetrievalChain.from_llm(
        llm=llm,
        retriever=retriever,
        return_source_documents=True,
        memory=memory,
        get_chat_history=lambda a: a,
        max_tokens_limit=4096
    )

    # Need to copy these otherwise various chains will feature templates that
    # point to the same objects.
    chain.combine_docs_chain.llm_chain.prompt = \
        chain.combine_docs_chain.llm_chain.prompt.copy()
    chain.combine_docs_chain.document_prompt = \
        chain.combine_docs_chain.document_prompt.copy()

    # Language mismatch fix:
    if lang_prompt_fix:
        chain.combine_docs_chain.llm_chain.prompt.template = \
            "Use the following pieces of context to answer the question at the end " \
            "in the same language as the question. If you don't know the answer, " \
            "just say that you don't know, don't try to make up an answer.\n\n" \
            "{context}\n\n" \
            "Question: {question}\n" \
            "Helpful Answer: "

    # Poor contexts fix using prompts:
    elif context_prompt_fix:
        chain.combine_docs_chain.llm_chain.prompt.template = \
            "Use only the relevant contexts to answer the question at the end " \
            ". Some pieces of context may not be relevant. If you don't know the answer, " \
            "just say that you don't know, don't try to make up an answer.\n\n" \
            "Contexts: \n{context}\n\n" \
            "Question: {question}\n" \
            "Helpful Answer: "
        chain.combine_docs_chain.document_prompt.template = "\tContext: {page_content}"

    # Trulens instrumentation.
    tc = Tru().Chain(chain=chain, feedbacks=feedbacks, verbose=True)

    return tc

In [ ]:
feedbacks=[
    f_toxic, 
    f_lang_match, 
    # f_relevance, 
    f_qs_relevance
]


In [ ]:
tc = new_conversation(
    feedbacks=feedbacks,
    #context_filter_fix=True
)

In [ ]:
res, record = tc.call_with_record("Who is Shayak?")